In [1]:
#Chargement des données
# Data
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os 
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import matplotlib.pyplot as plt
import math
# Graphics
import seaborn as sns ; sns.set()

from sklearn.experimental import enable_halving_search_cv # noqa

from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import FunctionTransformer
# from sklearn.metrics import root_mean_squared_error

In [2]:
load_dotenv('BDD_URL.env')
BDD_URL = os.environ['BDD_URL']
engine = create_engine(BDD_URL)

SQL_filtre= """
SET search_path to principal;
SELECT *
from "filmview"
where 'Comedy' = ANY(string_to_array("genres", ','))
limit 10000;
"""
SQL= """
SET search_path to principal;
SELECT  "primaryTitle", "titleType", "isAdult", "startYear", "runtimeMinutes", "genres", "averageRating", "directors", "writers", "actor", "producer", "cinematographer"
from "castview"
where "runtimeMinutes" Is NOT null and "titleType" = 'movie' and "averageRating" is NOT NULL and "genres" is NOT NULL and "startYear" is NOT NULL and "isAdult" is NOT NULL
limit 10000;
"""
df = pd.read_sql(SQL, engine)
engine.dispose()
df

,primaryTitle,titleType,isAdult,startYear,runtimeMinutes,genres,averageRating,directors,writers,actor,producer,cinematographer
0,Miss Jerry,movie,0,1894,45,Romance,5.3,Alexander_Black,Alexander_Black,"Blanche_Bayliss,Chauncey_Depew,William_Courtenay",None,None
1,The Corbett-Fitzsimmons Fight,movie,0,1897,100,"Documentary,News,Sport",5.3,Enoch_J._Rector,None,James_J._Corbett,None,None
2,The Story of the Kelly Gang,movie,0,1906,70,"Action,Adventure,Biography",6.0,Charles_Tait,Charles_Tait,"Elizabeth_Tait,John_Tait,Nicholas_Brierley,Nor...","Nevin_Tait,W.A._Gibson",None
3,The Prodigal Son,movie,0,1907,90,Drama,5.0,Michel_Carré,Michel_Carré,"Christiane_Mandelys,Georges_Wague,Gilberte_Ser...",None,None
4,The Fairylogue and Radio-Plays,movie,0,1908,120,"Adventure,Fantasy",5.0,"Francis_Boggs,Otis_Turner","L._Frank_Baum,Otis_Turner","Frank_Burns,George_E._Wilson,Wallace_Illington",None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,"Jalisco, Don't Backslide",movie,0,1941,100,"Comedy,Drama,Musical",7.2,Joselito_Rodríguez,"Aurelio_Robles_Castillo,Ismael_Rodríguez,Josel...","Ángel_Garasa,Carlos_López,Gloria_Marín,Jorge_N...",None,None
9996,Babes on Broadway,movie,0,1941,118,"Comedy,Family,Musical",6.6,Busby_Berkeley,"Elaine_Ryan,Fred_F._Finklehoffe","Fay_Bainter,Mickey_Rooney,Virginia_Weidler",None,None
9997,Baby Face Morgan,movie,0,1942,63,"Comedy,Crime,Romance",5.2,Arthur_Dreifuss,"Edward_Dein,Jack_Rubin,Oscar_Brodney","Chick_Chandler,Mary_Carlisle,Richard_Cromwell,...",None,None
9998,Back-Room Boy,movie,0,1942,82,"Comedy,Mystery",6.2,Herbert_Mason,"J.O.C._Orton,Marriott_Edgar,Val_Guest","Arthur_Askey,Googie_Withers,Graham_Moffatt,Moo...",Edward_Black,None


In [3]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   primaryTitle     10000 non-null  object 
 1   titleType        10000 non-null  object 
 2   isAdult          10000 non-null  int64  
 3   startYear        10000 non-null  int64  
 4   runtimeMinutes   10000 non-null  int64  
 5   genres           10000 non-null  object 
 6   averageRating    10000 non-null  float64
 7   directors        9989 non-null   object 
 8   writers          9920 non-null   object 
 9   actor            9946 non-null   object 
 10  producer         381 non-null    object 
 11  cinematographer  1015 non-null   object 
dtypes: float64(1), int64(3), object(8)
memory usage: 937.6+ KB


In [4]:
def liste_en_texte(lst):
    if isinstance(lst, list):
        return ' '.join(lst)
    else:
        return lst

In [5]:

df['genres']=df['genres'].str.replace(',', ' ')
df['genres'].fillna('missing', inplace=True)



# df['actor']=df['actor'].apply(liste_en_texte)
# df['actor'].fillna('missing', inplace=True)

df['actor'].fillna('missing', inplace=True)
df['actor']=df['actor'].str.replace(',', ' ')

df['directors'].fillna('missing', inplace=True)
df['directors']=df['directors'].str.replace(',', ' ')

df['writers'].fillna('missing', inplace=True)
df['writers']=df['writers'].str.replace(',', ' ')


# df['producer']=df['producer'].apply(liste_en_texte)
# df['producer'].fillna('missing', inplace=True)

df['producer'].fillna('missing', inplace=True)
df['producer']=df['producer'].str.replace(',', ' ')

# df['cinematographer']=df['cinematographer'].apply(liste_en_texte)
# df['cinematographer'].fillna('missing', inplace=True)

df['cinematographer'].fillna('missing', inplace=True)
df['cinematographer']=df['cinematographer'].str.replace(',', ' ')

df['primaryTitle'].fillna('missing', inplace=True)
df['primaryTitle']=df['primaryTitle'].str.replace(',', ' ')

df['titleType'].fillna('missing', inplace=True)
df['titleType']=df['titleType'].str.replace(',', ' ')



In [6]:
df

,primaryTitle,titleType,isAdult,startYear,runtimeMinutes,genres,averageRating,directors,writers,actor,producer,cinematographer
0,Miss Jerry,movie,0,1894,45,Romance,5.3,Alexander_Black,Alexander_Black,Blanche_Bayliss Chauncey_Depew William_Courtenay,missing,missing
1,The Corbett-Fitzsimmons Fight,movie,0,1897,100,Documentary News Sport,5.3,Enoch_J._Rector,missing,James_J._Corbett,missing,missing
2,The Story of the Kelly Gang,movie,0,1906,70,Action Adventure Biography,6.0,Charles_Tait,Charles_Tait,Elizabeth_Tait John_Tait Nicholas_Brierley Nor...,Nevin_Tait W.A._Gibson,missing
3,The Prodigal Son,movie,0,1907,90,Drama,5.0,Michel_Carré,Michel_Carré,Christiane_Mandelys Georges_Wague Gilberte_Ser...,missing,missing
4,The Fairylogue and Radio-Plays,movie,0,1908,120,Adventure Fantasy,5.0,Francis_Boggs Otis_Turner,L._Frank_Baum Otis_Turner,Frank_Burns George_E._Wilson Wallace_Illington,missing,missing
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Jalisco Don't Backslide,movie,0,1941,100,Comedy Drama Musical,7.2,Joselito_Rodríguez,Aurelio_Robles_Castillo Ismael_Rodríguez Josel...,Ángel_Garasa Carlos_López Gloria_Marín Jorge_N...,missing,missing
9996,Babes on Broadway,movie,0,1941,118,Comedy Family Musical,6.6,Busby_Berkeley,Elaine_Ryan Fred_F._Finklehoffe,Fay_Bainter Mickey_Rooney Virginia_Weidler,missing,missing
9997,Baby Face Morgan,movie,0,1942,63,Comedy Crime Romance,5.2,Arthur_Dreifuss,Edward_Dein Jack_Rubin Oscar_Brodney,Chick_Chandler Mary_Carlisle Richard_Cromwell ...,missing,missing
9998,Back-Room Boy,movie,0,1942,82,Comedy Mystery,6.2,Herbert_Mason,J.O.C._Orton Marriott_Edgar Val_Guest,Arthur_Askey Googie_Withers Graham_Moffatt Moo...,Edward_Black,missing


In [7]:
# Séparation des caractéristiques et de la cible
X = df.drop(columns=["averageRating"])
# X = df[['genres','actor']]
y = df["averageRating"]

# Séparation des données d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [8]:
# Définition des colonnes numériques, textuelles et de description
numeric_features = ['startYear', 'runtimeMinutes']
boolean_features = 'isAdult'
text_features = ['titleType']

In [9]:
# Création des transformers pour les colonnes numériques, booléennes, textuelles et de description
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())
])

boolean_transformer = FunctionTransformer(lambda x: x.astype(bool).values.reshape(-1, 1)) 

text_transformer = Pipeline(steps=[
    # ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

Vect_transformer = Pipeline([
    # ('imputer', SimpleImputer(strategy='constant', fill_value='missing')), 
    ('vect', CountVectorizer(decode_error='ignore', analyzer='word', ngram_range=(2, 2))) #max_features=1000, analyzer="word"
])
tfidf_transformer = Pipeline([
    # ('imputer', SimpleImputer(strategy='constant', fill_value='missing')), 
    ('tf_idf', TfidfVectorizer(decode_error='ignore', analyzer='word', ngram_range=(2, 2))) #max_features=1000, sublinear_tf=True
])
# Création d'un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('bool', boolean_transformer, boolean_features),
        ('text', text_transformer, text_features),
        ('title', tfidf_transformer, 'primaryTitle'),
        ('genres', tfidf_transformer, 'genres'),
        ('actor', tfidf_transformer, 'actor'),
        ('directors', tfidf_transformer, 'directors'),
        ('producer', tfidf_transformer, 'producer'),
        ('cinematographer', tfidf_transformer, 'cinematographer'),
        ('writers', tfidf_transformer, 'writers')
    ])
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', RobustScaler())]),
                                 ['startYear', 'runtimeMinutes']),
                                ('bool',
                                 FunctionTransformer(func=<function <lambda> at 0x000002392F6B9EE0>),
                                 'isAdult'),
                                ('text',
                                 Pipeline(steps=[('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['titleType']),
                                ('title',
                                 Pipeli...
                                 'directors'),
                                ('producer',
                                 Pipeline(steps=[('tf_idf',
                                                  TfidfVectorizer(decode_error='ignore',
                                                                  ngram_range=(2,
                                                                               2)))]),
                                 'producer'),
                                ('cinematographer',
                                 Pipeline(steps=[('tf_idf',
                                                  TfidfVectorizer(decode_error='ignore',
                                                                  ngram_range=(2,
                                                                               2)))]),
                                 'cinematographer'),
                                ('writers',
                                 Pipeline(steps=[('tf_idf',
                                                  TfidfVectorizer(decode_error='ignore',
                                                                  ngram_range=(2,
                                                                               2)))]),
                                 'writers')])

In [10]:
# Création des pipelines pour chaque modèle
pipelines = {
    'Linear Regression': Pipeline([('preprocessor', preprocessor), ('regressor', LinearRegression())]),
    'Ridge Regression': Pipeline([('preprocessor', preprocessor), ('regressor', Ridge())]),
    'Lasso Regression': Pipeline([('preprocessor', preprocessor), ('regressor', Lasso())]),
    'ElasticNet': Pipeline([('preprocessor', preprocessor), ('regressor', ElasticNet())]),
    'Random Forest Regression': Pipeline([('preprocessor', preprocessor), ('regressor', RandomForestRegressor())]),
    'Gradient Boosting Regression': Pipeline([('preprocessor', preprocessor), ('regressor', GradientBoostingRegressor())]),
    'SVR': Pipeline([('preprocessor', preprocessor), ('regressor', SVR())]),
}

# Paramètres pour GridSearchCV pour chaque modèle
parameters = {
    'Linear Regression': {'regressor__fit_intercept': [True,False]},
    'Ridge Regression': {'regressor__alpha': [0.1, 1.0, 5.0,10]},
    'Lasso Regression': {'regressor__alpha': [0.1, 1.0, 5.0,10]},
    'ElasticNet': {'regressor__alpha': [0.1, 1.0, 5.0,10], 'regressor__l1_ratio': [0.1, 0.5, 0.9]},
    'Random Forest Regression': {'regressor__n_estimators': [150,200], 'regressor__max_depth': [50,100]},# None, 
    'Gradient Boosting Regression': {'regressor__n_estimators': [150,200], 'regressor__max_depth': [50, 100]},
    'SVR': {'regressor__kernel': ['linear', 'rbf'], 'regressor__C': [0.1, 1.0, 10.0]},
}

# Scoring : RMSE, R2 et MAE
scoring = {'RMSE': 'neg_root_mean_squared_error',
           'R2': 'r2',
           'MAE': 'neg_mean_absolute_error'}


In [11]:
def Grid(X_train, y_train, pipeline, parameters, cv=5):
  # Scoring
  #multi_scoring = {mean_squared_error,r2_score}
    
    
  # Grid search
  grid = GridSearchCV(pipeline, parameters,  scoring=scoring, refit='RMSE', cv=cv, n_jobs =-1, verbose = 1)

  # Fit
  grid.fit(X_train, y_train)

  # Scores and results
  best_score = grid.best_score_.round(4)
  best_params = grid.best_params_
  training_time = grid.cv_results_['mean_fit_time'].mean().round(4)

  # Output
  return({
      'best_score': best_score,
      'best_params': best_params,
      'training_time': training_time,
      'fitted_model': grid.best_estimator_
  })

In [12]:
def afficheResults (grid):
    model_name = grid['fitted_model'].named_steps['regressor'].__class__.__name__
    print(f"{model_name} training time: {grid['training_time']}")
    print(f"Best {model_name} parameters: {grid['best_params']}")
    print(f"Best {model_name} score: {-grid['best_score']}")
 
    
    

In [13]:
X_train

,primaryTitle,titleType,isAdult,startYear,runtimeMinutes,genres,directors,writers,actor,producer,cinematographer
9254,Billy the Kid's Range War,movie,0,1941,57,Drama Western,Sam_Newfield,William_Lively,Al_St._John Bob_Steele Carleton_Young Joan_Bar...,missing,missing
1561,Madame Doesn't Want Children,movie,0,1926,98,Drama,Alexander_Korda,Adolf_Lantz Béla_Balázs Clément_Vautel,Harry_Liedtke María_Corda Maria_Paudler Trude_...,missing,missing
1670,Sparrows,movie,0,1926,109,Drama,Tom_McNamara William_Beaudine,C._Gardner_Sullivan George_Marion_Jr. Winifred...,Gustav_von_Seyffertitz Mary_Louise_Miller Mary...,missing,missing
6087,By Candlelight,movie,0,1933,70,Comedy Music Romance,James_Whale,F._Hugh_Herbert Hanns_Kräly Karen_DeWolf Karl_...,Dorothy_Revier Elissa_Landi Nils_Asther Paul_L...,missing,missing
6669,Paradise for Two,movie,0,1937,77,Comedy Music,Thornton_Freeland,Arthur_Macrae Robert_Liebmann Robert_Stevenson...,Arthur_Riscoe Googie_Withers Jack_Hulbert Patr...,missing,missing
...,...,...,...,...,...,...,...,...,...,...,...
5734,Mason of the Mounted,movie,0,1932,58,Drama Western,Harry_L._Fraser,Harry_L._Fraser,Andy_Shuford Bill_Cody Jack_Carlyle James_A._M...,missing,missing
5191,It's a Small World,movie,0,1935,70,Comedy Romance,Irving_Cummings,Albert_Treynor Gladys_Lehman Sam_Hellman,Irving_Cummings Raymond_Walburn Spencer_Tracy ...,missing,missing
5390,The Woman Between,movie,0,1931,73,Drama,Victor_Schertzinger,Howard_Estabrook Irving_Kaye_Davis,Lili_Damita Miriam_Seegar O.P._Heggie,missing,missing
860,Kit Carson Over the Great Divide,movie,0,1925,62,Western,Frank_S._Mattison,Frank_S._Mattison J.C._Hull,Henry_B._Walthall Marguerite_Snow Roy_Stewart ...,missing,missing


In [14]:
y_train.info()

<class 'pandas.core.series.Series'>
Index: 8000 entries, 9254 to 7270
Series name: averageRating
Non-Null Count  Dtype  
--------------  -----  
8000 non-null   float64
dtypes: float64(1)
memory usage: 125.0 KB


In [15]:
# Boucle sur les modèles pour ajuster avec GridSearchCV
models = {}

# Boucle sur les modèles pour ajuster avec GridSearchCV et évaluation
for model_name, pipeline in pipelines.items():
    print(f"\n..............{model_name}..............................")
    grid_search = Grid(X_train, y_train, pipeline, parameters[model_name], cv=5)
    afficheResults (grid_search)
    
    best_model = grid_search['fitted_model']
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    Rmse = round(math.sqrt(mse), 4)
    models[model_name] = [best_model,Rmse]
    print(f"{model_name} RMSE: {Rmse}")


..............Linear Regression..............................
Fitting 5 folds for each of 2 candidates, totalling 10 fits
LinearRegression training time: 2.6737
Best LinearRegression parameters: {'regressor__fit_intercept': False}
Best LinearRegression score: 0.9527
Linear Regression RMSE: 0.9772

..............Ridge Regression..............................
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Ridge training time: 1.128
Best Ridge parameters: {'regressor__alpha': 5.0}
Best Ridge score: 0.8581
Ridge Regression RMSE: 0.8973

..............Lasso Regression..............................
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Lasso training time: 1.6846
Best Lasso parameters: {'regressor__alpha': 0.1}
Best Lasso score: 0.8976
Lasso Regression RMSE: 0.94

..............ElasticNet..............................
Fitting 5 folds for each of 12 candidates, totalling 60 fits


KeyboardInterrupt: 

In [ ]:
'''
# Boucle sur les modèles pour ajuster avec GridSearchCV
results = {}

# Boucle sur les modèles pour ajuster avec GridSearchCV et évaluation
for model_name, pipeline in pipelines.items():
    print(f"Training {model_name}...")
    grid_search = GridSearchCV(pipeline, parameters[model_name],  scoring=scoring, refit='RMSE', cv=5, n_jobs =-1, verbose = 1)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    mse = root_mean_squared_error(y_test, y_pred)
    results[model_name] = mse
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best {model_name} score: {-grid_search.best_score_}")
    print(f"{model_name} MSE: {mse}")
    print()
'''

'\n# Boucle sur les modèles pour ajuster avec GridSearchCV\nresults = {}\n\n# Boucle sur les modèles pour ajuster avec GridSearchCV et évaluation\nfor model_name, pipeline in pipelines.items():\n    print(f"Training {model_name}...")\n    grid_search = GridSearchCV(pipeline, parameters[model_name],  scoring=scoring, refit=\'RMSE\', cv=5, n_jobs =-1, verbose = 1)\n    grid_search.fit(X_train, y_train)\n    best_model = grid_search.best_estimator_\n    y_pred = best_model.predict(X_test)\n    mse = root_mean_squared_error(y_test, y_pred)\n    results[model_name] = mse\n    print(f"Best parameters: {grid_search.best_params_}")\n    print(f"Best {model_name} score: {-grid_search.best_score_}")\n    print(f"{model_name} MSE: {mse}")\n    print()\n'

In [ ]:
'''
# Evaluation des modèles sur les données de test
print("\nComparaison des performances des modèles sur les données de test:")
for model_name, mse in results.items():
    print(f"{model_name}: MSE = {mse}")
    
'''

'\n# Evaluation des modèles sur les données de test\nprint("\nComparaison des performances des modèles sur les données de test:")\nfor model_name, mse in results.items():\n    print(f"{model_name}: MSE = {mse}")\n    \n'